# Section 3: Customer Metrics System Design

The CEO needs weekly metrics and the sales team needs real-time insights. We have Postgres (user events), Stripe (billing), and Sentry (errors), but no unified analytics system yet.

#### Database Schema Assumption

Table: events_big (Postgres)

Example: A user clicks "Submit" button on the note editor page
- event_id (String): "evt_123abc" - Unique identifier for this click event
- org_id (String): "550e8400-e29b-41d4-a716-446655440000" - Organization "Acme Medical"
- provider_id (String, Nullable): "7b44b47b-62d7-4fd6-8ef3-984f251c5c3a" - Dr. Smith
- user_id (String, Nullable): "a51234bc-82f1-4c39-92b9-1234abcd5678" - Nurse Johnson
- occurred_at (String): "2023-12-01 13:45:23.456" - When click happened
- event_name (String): "note_submit" - Type of event
- session_id (String): "sess_xyz789" - Browser session
- request_id (String): "req_456def" - API request trace ID
- source (String): "app" - From web application
- endpoint (String): "/api/v1/notes" - API endpoint called
- device (String): "Chrome/Windows" - User's browser/OS
- geo_country (String): "US" - User location
- properties (String): {"note_id": "123", "length": 500} - Event details
- inserted_at (String): "2023-12-01 13:45:23.789" - DB write time
- _ingest_source (String): "frontend_logger" - Data collector
- _ingest_ts (String): "2023-12-01 13:45:24.123" - Collection time

Table: stripe_billing_big (Stripe)

Example: A customer's monthly subscription invoice
- invoice_id (String): "in_1234" - Unique invoice identifier
- org_id (UUID): "550e8400-e29b-41d4-a716-446655440000" - Organization "Acme Medical"
- customer_id (String): "cus_xyz789" - Stripe customer ID
- subscription_id (String, Nullable): "sub_abc123" - Active subscription
- price_id (String, Nullable): "price_123" - Product price point
- product (String, Nullable): "Enterprise Plan" - Product name
- currency (String): "USD" - Invoice currency
- period_start (Timestamp): "2023-12-01" - Billing period start
- period_end (Timestamp): "2023-12-31" - Billing period end
- amount_cents (Integer): 100000 - Total amount ($1000.00)
- subtotal_cents (Integer, Nullable): 95000 - Pre-tax amount
- tax_cents (Integer, Nullable): 5000 - Tax amount
- discount_cents (Integer, Nullable): 0 - Discounts applied
- quantity (Integer, Nullable): 10 - Number of seats
- status (String): "paid" - Invoice status
- paid_at (Timestamp, Nullable): "2023-12-01 00:00:00" - Payment timestamp
- refunded_cents (Integer, Nullable): 0 - Amount refunded
- mrr_cents (Integer, Nullable): 100000 - Monthly recurring revenue
- churn_flag (Boolean, Nullable): false - Churned this period
- expansion_flag (Boolean, Nullable): true - Seats increased
- _ingest_source (String): "stripe_webhook" - Data source
- _ingest_ts (Timestamp): "2023-12-01 00:00:01" - Ingestion time

Table: sentry_errors_big (Sentry)

Example: An error occurs in the note editor
- event_id (String): "err_123abc" - Unique error event identifier
- org_id (UUID, Nullable): "550e8400-e29b-41d4-a716-446655440000" - Organization mapped from project
- provider_id (UUID, Nullable): "7b44b47b-62d7-4fd6-8ef3-984f251c5c3a" - Provider if linked
- project (String): "clinical-notes-web" - Sentry project name
- group_id (String, Nullable): "grp_xyz789" - Error grouping fingerprint
- level (String): "error" - Severity level (error/warning/info)
- message (String): "Failed to save note" - Error summary
- culprit (String, Nullable): "NoteEditor.save()" - Error source
- release (String, Nullable): "v1.2.3" - Software version
- environment (String, Nullable): "production" - Runtime environment
- url (String, Nullable): "/notes/123/edit" - Related URL
- user_id_raw (String, Nullable): "user_456def" - Reported user ID
- created_at (Timestamp): "2023-12-01 13:45:23.456" - When error occurred
- tags (JSON, Nullable): {"browser": "Chrome"} - Error tags
- contexts (JSON, Nullable): {"os": "Windows"} - Error context
- group_first_seen (Timestamp, Nullable): "2023-11-15" - First occurrence in group
- group_last_seen (Timestamp, Nullable): "2023-12-01" - Latest occurrence
- group_count (Integer, Nullable): 42 - Total occurrences in group
- _ingest_source (String): "sentry_webhook" - Data source
- _ingest_ts (Timestamp): "2023-12-01 13:45:24.123" - Ingestion time

# 3.1.1 Customer Health Metrics

Assume tables: organizations, providers, summarization_jobs
- (1) normalizing shared IDs, then 
- (2) modeling a dimension/fact layer where summarization_jobs is the fact built from app events and error signals, while Stripe backfills org-level attributes.

organizations:
- org_id (UUID, PK)
- org_name (TEXT, nullable if unknown)
- stripe_customer_id (TEXT)
- current_plan (TEXT)
- seats_purchased (INT)
- mrr_cents (INT)
- is_active (BOOL)
- first_seen_at, last_seen_at (TIMESTAMP)

providers：
- provider_id (UUID, PK)
- org_id (UUID, FK → organizations)
- role (TEXT, e.g., "doctor", "nurse")
- first_seen_at, last_seen_at (TIMESTAMP)
- last_device (TEXT), last_geo_country (TEXT)

summarization_jobs：
- job_id (TEXT, PK) — e.g., coalesce(properties.note_id, request_id, event_id)
- org_id (UUID, FK)
- provider_id (UUID, FK, nullable)
- request_id (TEXT)
- note_id (TEXT, nullable)
- session_id (TEXT, nullable)
- started_at (TIMESTAMP) — from first app event (e.g., note_submit)
- completed_at (TIMESTAMP, nullable) — inferred if a later success event exists
- failed_at (TIMESTAMP, nullable) — inferred from Sentry error
- patient_status (TEXT: 'Locked'/'Review')
- source (TEXT) — "app", "api"
- endpoint (TEXT)
- client_release (TEXT) — from Sentry release if available
- duration_ms (INT, nullable) — completed_at - started_at
- error_group_id (TEXT, nullable) — Sentry group_id
- error_level (TEXT, nullable)


Define "active": A provider who has successfully completed at least one summarization job 
during the week. This means:
- They have at least one record in summarization_jobs
- With status = 'success'
- Created within the 7-day window being measured
- Multiple jobs in the same week still count as one active provider

In [ ]:
-- Organizations
CREATE TABLE organizations (
  org_id uuid PRIMARY KEY,
  org_name text NOT NULL,
  stripe_customer_id text,
  current_plan text,
  seats_purchased int,
  mrr_cents int,
  is_active boolean DEFAULT true,
  first_seen_at timestamptz,
  last_seen_at timestamptz
);

-- Providers
CREATE TABLE providers (
  provider_id uuid PRIMARY KEY,
  org_id uuid NOT NULL REFERENCES organizations(org_id),
  role text,                         -- 'doctor','nurse', etc.
  first_seen_at timestamptz,
  last_seen_at timestamptz,
  last_device text,
  last_geo_country text
);

-- Summarization jobs (fact)
CREATE TABLE summarization_jobs (
  job_id text PRIMARY KEY,
  org_id uuid NOT NULL REFERENCES organizations(org_id),
  provider_id uuid REFERENCES providers(provider_id),
  request_id text,
  note_id text,
  session_id text,
  status text NOT NULL CHECK (status IN ('success','error')),
  feature text,                      -- 'note','auto_coding', ...
  created_at timestamptz NOT NULL,   -- enqueue/start time
  completed_at timestamptz,          -- completed time (NULL if not finished)
  failed_at timestamptz,
  duration_ms int,                   -- derived: completed_at - created_at
  patient_status text CHECK (patient_status IN ('Locked','Review')),
  source text,                       -- 'app','api'
  endpoint text,
  client_release text,
  error_group_id text,
  error_level text
);

-- App/events (wide)
CREATE TABLE events_big (
  event_id text PRIMARY KEY,
  org_id uuid,                 -- lower() cast upstream if needed
  provider_id uuid,
  user_id uuid,
  occurred_at timestamptz NOT NULL,
  event_name text,
  session_id text,
  request_id text,
  source text,
  endpoint text,
  device text,
  geo_country text,
  properties jsonb,
  inserted_at timestamptz,
  _ingest_source text,
  _ingest_ts timestamptz
);

-- Stripe invoices (wide)
CREATE TABLE stripe_billing_big (
  invoice_id text PRIMARY KEY,
  org_id uuid,                   -- join via customer mapping, or denormalized
  customer_id text,
  subscription_id text,
  price_id text,
  product text,
  currency text,
  period_start timestamptz,
  period_end timestamptz,
  amount_cents int,
  subtotal_cents int,
  tax_cents int,
  discount_cents int,
  quantity int,
  status text,                   -- 'paid','void','open',...
  paid_at timestamptz,
  refunded_cents int,
  mrr_cents int,
  churn_flag boolean,
  expansion_flag boolean,
  _ingest_source text,
  _ingest_ts timestamptz
);

-- Sentry errors (wide)
CREATE TABLE sentry_errors_big (
  event_id text PRIMARY KEY,
  org_id uuid,                   -- coalesced/mapped upstream
  provider_id uuid,
  project text,
  group_id text,
  level text,                    -- 'error','fatal','warning'
  message text,
  culprit text,
  release text,
  environment text,
  url text,
  user_id_raw text,
  created_at timestamptz NOT NULL,
  tags jsonb,
  contexts jsonb,
  group_first_seen timestamptz,
  group_last_seen timestamptz,
  group_count int,
  _ingest_source text,
  _ingest_ts timestamptz
);

-- Recommended indexes
CREATE INDEX IF NOT EXISTS idx_sj_status_completed_org
  ON summarization_jobs (status, completed_at, org_id);

CREATE INDEX IF NOT EXISTS idx_sj_provider_completed
  ON summarization_jobs (provider_id, completed_at);

CREATE INDEX IF NOT EXISTS idx_events_org_session_time
  ON events_big (org_id, session_id, occurred_at);

CREATE INDEX IF NOT EXISTS idx_sentry_org_time
  ON sentry_errors_big (org_id, created_at);

CREATE INDEX IF NOT EXISTS idx_stripe_org_period
  ON stripe_billing_big (org_id, period_start, period_end);

Weekly active providers per org

In [ ]:
WITH weekly_jobs AS (
  SELECT
    sj.org_id,
    DATE_TRUNC('week', sj.completed_at AT TIME ZONE 'America/Los_Angeles') AS week_start,
    sj.provider_id
  FROM summarization_jobs sj
  WHERE sj.status = 'success' AND sj.provider_id IS NOT NULL AND sj.completed_at IS NOT NULL
)
SELECT
  o.org_id, o.org_name, week_start::date AS week_start,
  COUNT(DISTINCT provider_id) AS weekly_active_providers
FROM weekly_jobs w
JOIN organizations o ON o.org_id = w.org_id
GROUP BY o.org_id, o.org_name, week_start
ORDER BY week_start, o.org_id;

Identify accounts with declining usage:
- Rule: Weekly Active Providers (WAP) decreased by ≥20% compared to previous week
- Example: If last week had 100 WAP and this week has 75 WAP, that's a 25% decline

In [ ]:
WITH weekly_wap AS (  -- step 1: per-org weekly active providers
  SELECT
    sj.org_id,
    DATE_TRUNC('week', sj.completed_at AT TIME ZONE 'America/Los_Angeles') AS week_start, 
    COUNT(DISTINCT sj.provider_id) AS wap
  FROM summarization_jobs sj
  WHERE sj.status = 'success'
    AND sj.provider_id IS NOT NULL
    AND sj.completed_at IS NOT NULL
    -- Optional window:
    -- AND sj.completed_at >= :start_date AND sj.completed_at < :end_date
  GROUP BY 1, 2
),
with_prev AS (  -- step 2: bring last week's WAP
  SELECT
    w.org_id,
    w.week_start::date AS week_start,
    w.wap,
    LAG(w.wap) OVER (PARTITION BY w.org_id ORDER BY w.week_start) AS prev_wap
  FROM weekly_wap w
)
SELECT
  o.org_id,
  o.org_name,
  wp.week_start,
  wp.prev_wap,
  wp.wap AS curr_wap,
  (wp.prev_wap - wp.wap) AS change_abs,                              
  ROUND(((wp.prev_wap - wp.wap)::numeric / wp.prev_wap) * 100, 1) AS decline_pct 
FROM with_prev wp
JOIN organizations o ON o.org_id = wp.org_id
WHERE wp.prev_wap > 0
  AND ((wp.prev_wap - wp.wap)::numeric / wp.prev_wap) >= 0.20        
ORDER BY wp.week_start, o.org_id;


Compare current month vs previous month activity

In [ ]:
-- Compare current month (MTD) vs previous month
WITH monthly AS (
  SELECT
    sj.org_id,
    DATE_TRUNC('month', sj.completed_at AT TIME ZONE 'America/Los_Angeles') AS month_start,
    COUNT(DISTINCT sj.provider_id) AS wap  -- Monthly Active Providers
  FROM summarization_jobs sj
  WHERE sj.status = 'success'
    AND sj.provider_id IS NOT NULL
    AND sj.completed_at IS NOT NULL
  GROUP BY 1, 2
),
bounds AS (
  SELECT
    DATE_TRUNC('month', NOW() AT TIME ZONE 'America/Los_Angeles') AS curr_month_start,
    DATE_TRUNC('month', (NOW() AT TIME ZONE 'America/Los_Angeles') - INTERVAL '1 month') AS prev_month_start
),
curr AS (
  SELECT m.org_id, m.wap
  FROM monthly m JOIN bounds b ON m.month_start = b.curr_month_start
),
prev AS (
  SELECT m.org_id, m.wap
  FROM monthly m JOIN bounds b ON m.month_start = b.prev_month_start
)
SELECT
  o.org_id,
  o.org_name,
  COALESCE(c.wap, 0) AS curr_wap,          -- Current month active providers (MTD)
  COALESCE(p.wap, 0) AS prev_wap,          -- Previous month active providers
  (COALESCE(c.wap,0) - COALESCE(p.wap,0)) AS change_abs,  -- Absolute change
  ROUND( (COALESCE(c.wap,0) - COALESCE(p.wap,0))::numeric
         / NULLIF(p.wap,0) * 100, 1)       AS change_pct   -- % change; NULLIF for null/zero division protection
FROM organizations o
LEFT JOIN curr c USING (org_id)
LEFT JOIN prev p USING (org_id)
WHERE c.wap IS NOT NULL OR p.wap IS NOT NULL
ORDER BY change_pct ASC, o.org_id;


# 3.1.2 Usage Pattern Analysis

Peak usage hours across all customers:
- Identify recurring peaks (e.g., Mon–Thu, 9 AM–1 PM PT).
- Plan auto-scaling policies to add capacity before these windows.
- Correlate with summarization_jobs to capture compute-heavy times separately from general events

In [ ]:
-- Average hourly load by day-of-week (0=Sunday)
WITH per_day_hour AS (
  SELECT
    DATE(occurred_at AT TIME ZONE 'America/Los_Angeles')                               AS d,      -- calendar day 日
    EXTRACT(DOW  FROM occurred_at AT TIME ZONE 'America/Los_Angeles')::int            AS dow,    -- day of week 周几
    EXTRACT(HOUR FROM occurred_at AT TIME ZONE 'America/Los_Angeles')::int            AS hour_of_day, -- 小时
    COUNT(*)                                                                           AS events
  FROM events_big
  WHERE occurred_at IS NOT NULL
  GROUP BY 1,2,3
)
SELECT
  dow,
  hour_of_day,
  ROUND(AVG(events)::numeric, 1) AS avg_events  -- average per (dow,hour) across days 跨天平均
FROM per_day_hour
GROUP BY dow, hour_of_day
ORDER BY dow, hour_of_day;


Target lowest activity windows for DB upgrades, deployments, or backups.

Check both events_big and summarization_jobs—sometimes general traffic is low but summarization jobs spike (or vice versa).

Consider customer timezone distribution: global customers may never have a full “quiet” window.

In [ ]:
-- Lowest activity windows (summarization jobs)
WITH hourly_jobs AS (
  SELECT
    DATE_TRUNC('hour', completed_at AT TIME ZONE 'America/Los_Angeles') AS hour_start,
    COUNT(*) AS jobs_count
  FROM summarization_jobs
  WHERE completed_at IS NOT NULL
    AND status = 'success'
  GROUP BY 1
)
SELECT *
FROM hourly_jobs
ORDER BY jobs_count ASC
LIMIT 5;  -- lowest 5 hours


Average notes per provider per org

In [ ]:
-- Average notes per provider per organization (optionally time-boxed)
WITH provider_notes AS (
  SELECT
    sj.org_id,
    sj.provider_id,
    COUNT(*) AS notes_count
  FROM summarization_jobs sj
  WHERE sj.status = 'success'
    AND sj.provider_id IS NOT NULL
    -- Optional time window:
    -- AND sj.completed_at >= :start_ts
    -- AND sj.completed_at <  :end_ts
  GROUP BY sj.org_id, sj.provider_id
),
org_averages AS (
  SELECT
    org_id,
    SUM(notes_count) AS total_notes,
    COUNT(*) AS providers_cnt,
    AVG(notes_count)::numeric(10,2) AS avg_notes_per_provider
  FROM provider_notes
  GROUP BY org_id
)
SELECT
  o.org_id,
  o.org_name,
  oa.providers_cnt,
  oa.total_notes,
  oa.avg_notes_per_provider
FROM org_averages oa
JOIN organizations o USING (org_id)
ORDER BY oa.avg_notes_per_provider DESC;


Orgs that might need training/support:
- High “Review” percentage – a large share of completed notes remain in “Review” status instead of “Locked.”
- High error rate – Sentry logs show frequent error or fatal events per number of attempts.
- Active billing – the customer has a recent paid invoice in Stripe.

In [ ]:
WITH status_by_org AS (            -- Last 30 days Review ratio (persisted patient_status)
  SELECT
    sj.org_id,
    COUNT(*) AS total_jobs,
    COUNT(*) FILTER (WHERE sj.patient_status = 'Review') AS review_cnt,
    COUNT(*) FILTER (WHERE sj.patient_status = 'Locked') AS locked_cnt,
    ROUND(
      COUNT(*) FILTER (WHERE sj.patient_status = 'Review')::numeric
      / NULLIF(COUNT(*), 0) * 100, 1
    ) AS review_pct
  FROM summarization_jobs sj
  WHERE sj.completed_at >= NOW() - INTERVAL '30 days'
  GROUP BY sj.org_id
),
errors_by_org AS (                 -- Last 30 days error rate (errors per 100 attempts)
  WITH sentry_30d AS (
    SELECT
      COALESCE(NULLIF(se.org_id, ''), NULL)::uuid AS org_id,
      COUNT(*) AS error_events
    FROM sentry_errors_big se
    WHERE se.created_at >= NOW() - INTERVAL '30 days'
      AND se.level IN ('error','fatal')
    GROUP BY 1
  ),
  usage_30d AS (
    SELECT
      LOWER(e.org_id)::uuid AS org_id,
      COUNT(*) AS attempt_events
    FROM events_big e
    WHERE e.occurred_at >= NOW() - INTERVAL '30 days'
      AND e.event_name IN ('note_submit','summarize_start','note_save')
    GROUP BY 1
  )
  SELECT
    u.org_id,
    u.attempt_events,
    COALESCE(s.error_events, 0) AS error_events,
    ROUND(COALESCE(s.error_events, 0)::numeric / NULLIF(u.attempt_events, 0) * 100, 1) AS error_perc_100
  FROM usage_30d u
  LEFT JOIN sentry_30d s USING (org_id)
),
stripe_active AS (                 -- Stripe active/paid signals
  SELECT
    sb.org_id,
    BOOL_OR(sb.status='paid' AND sb.period_end >= NOW() - INTERVAL '30 days') AS paid_recently,
    MAX(sb.mrr_cents) AS mrr_cents,
    MAX(sb.quantity) AS seats_purchased
  FROM stripe_billing_big sb
  GROUP BY sb.org_id
)
SELECT
  o.org_id,
  o.org_name,
  sa.paid_recently,
  sa.mrr_cents,
  sa.seats_purchased,
  sb.total_jobs,
  sb.review_cnt,
  sb.locked_cnt,
  sb.review_pct,
  eb.error_events,
  eb.attempt_events,
  eb.error_perc_100,
  (
    (sb.review_pct >= 50)            -- Rule 1: Review ratio >= 50%
    OR (eb.error_perc_100 >= 5)      -- Rule 2: Error rate >= 5/100 attempts
  ) AND COALESCE(sa.paid_recently, FALSE) AS needs_training_flag
FROM status_by_org sb
LEFT JOIN errors_by_org eb ON eb.org_id = sb.org_id
LEFT JOIN stripe_active sa ON sa.org_id = sb.org_id
LEFT JOIN organizations o ON o.org_id = sb.org_id
WHERE sb.total_jobs >= 20
ORDER BY needs_training_flag DESC, sb.review_pct DESC, eb.error_perc_100 DESC;


Customers struggling with integration：
- Session duration – Short or overly long sessions may indicate user struggle.
- Bounce rate – Sessions with only one event suggest integration barriers or drop-offs.
- Error-tainted sessions – Proportion of sessions containing Sentry errors.
- Success rate – Proportion of sessions that contain at least one successfully completed summarization job.

In [ ]:
-- Session health by org (last 30 days, PT)
WITH ev AS (
  SELECT
    lower(org_id)::uuid AS org_id,
    nullif(provider_id,'')::uuid AS provider_id,
    session_id,
    occurred_at AT TIME ZONE 'America/Los_Angeles' AS ts
  FROM events_big
  WHERE occurred_at >= NOW() - INTERVAL '30 days'
    AND session_id IS NOT NULL
),
sessions AS (  -- Session start/end times, event count, main provider, request_id list (if needed)
  SELECT
    org_id,
    session_id,
    -- If multiple providers in session, take provider from most recent event
    (ARRAY_REMOVE(ARRAY_AGG(provider_id ORDER BY ts DESC), NULL))[1] AS provider_id,
    MIN(ts) AS session_start,
    MAX(ts) AS session_end,
    EXTRACT(EPOCH FROM (MAX(ts) - MIN(ts)))::int AS duration_sec,
    COUNT(*) AS events_count
  FROM ev
  GROUP BY org_id, session_id
),
-- Successful session: At least one successful summarization_job within session window (same org and aligned provider)
success_map AS (
  SELECT
    s.org_id, s.session_id,
    EXISTS (
      SELECT 1
      FROM summarization_jobs sj
      WHERE sj.org_id = s.org_id
        AND sj.provider_id IS NOT DISTINCT FROM s.provider_id
        AND sj.status = 'success'
        AND sj.completed_at AT TIME ZONE 'America/Los_Angeles'
              BETWEEN s.session_start AND s.session_end
    ) AS has_success
  FROM sessions s
),
-- Error session: Sentry error occurs within session window (same org)
error_map AS (
  SELECT
    s.org_id, s.session_id,
    EXISTS (
      SELECT 1
      FROM sentry_errors_big se
      WHERE COALESCE(NULLIF(se.org_id,''), NULL)::uuid = s.org_id
        AND se.level IN ('error','fatal')
        AND se.created_at AT TIME ZONE 'America/Los_Angeles'
              BETWEEN s.session_start AND s.session_end
    ) AS has_error
  FROM sessions s
),
session_health AS (
  SELECT
    s.*,
    sm.has_success,
    em.has_error,
    (s.events_count = 1) AS is_bounce
  FROM sessions s
  LEFT JOIN success_map sm USING (org_id, session_id)
  LEFT JOIN error_map   em USING (org_id, session_id)
),
org_session_metrics AS (
  SELECT
    org_id,
    COUNT(*) AS total_sessions,
    -- Median/95th percentile session duration
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY duration_sec) AS p50_duration_sec,
    PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY duration_sec) AS p95_duration_sec,
    AVG(events_count)::numeric(10,2) AS avg_events_per_session,
    SUM(is_bounce::int) AS bounce_sessions,
    SUM(has_error::int) AS error_sessions,
    SUM(has_success::int) AS success_sessions
  FROM session_health
  GROUP BY org_id
)
SELECT
  o.org_id,
  o.org_name,
  m.total_sessions,
  m.p50_duration_sec,
  m.p95_duration_sec,
  m.avg_events_per_session,
  ROUND(m.bounce_sessions::numeric / NULLIF(m.total_sessions,0) * 100, 1) AS bounce_rate_pct,   -- Bounce rate
  ROUND(m.error_sessions::numeric  / NULLIF(m.total_sessions,0) * 100, 1) AS error_session_pct, -- Error session percentage
  ROUND(m.success_sessions::numeric/ NULLIF(m.total_sessions,0) * 100, 1) AS success_rate_pct   -- Success conversion rate
FROM org_session_metrics m
JOIN organizations o USING (org_id)
WHERE m.total_sessions >= 30  -- Sample size threshold
ORDER BY success_rate_pct ASC, p50_duration_sec DESC;


# 3.1.3 Product Adoption Funnel

Journey: signup → first_transcription → regular_usage

In [ ]:
WITH first_transcription AS (
  SELECT
    provider_id,
    MIN(created_at) AS first_tx_at
  FROM summarization_jobs
  WHERE status='success' AND feature='note'
  GROUP BY provider_id
),
weekly_ok AS (
  SELECT
    provider_id,
    date_trunc('week', created_at) AS wk,
    COUNT(*) FILTER (WHERE status='success' AND feature='note') AS ok_cnt
  FROM summarization_jobs
  GROUP BY provider_id, date_trunc('week', created_at)
),
regular_usage AS (
  SELECT DISTINCT w1.provider_id
  FROM weekly_ok w1
  JOIN weekly_ok w2
    ON w1.provider_id = w2.provider_id
   AND w2.wk = w1.wk + INTERVAL '1 week'
  WHERE w1.ok_cnt >= 3 AND w2.ok_cnt >= 3
)
SELECT
  o.org_id AS org_id, o.org_name,
  COUNT(*) FILTER (WHERE p.provider_id IS NOT NULL)           AS signups,
  COUNT(*) FILTER (WHERE ft.first_tx_at IS NOT NULL)          AS first_transcription,
  COUNT(*) FILTER (WHERE ru.provider_id IS NOT NULL)          AS regular_users
FROM organizations o
LEFT JOIN providers p            ON p.org_id = o.org_id
LEFT JOIN first_transcription ft ON ft.provider_id = p.provider_id
LEFT JOIN regular_usage ru       ON ru.provider_id = p.provider_id
GROUP BY o.org_id, o.org_name
ORDER BY signups DESC;


Define what "regular usage" means for medical practices：
- Weekly Active Providers
- Session Consistency
- Volume Alignment
- Workflow Completion

Feature adoption: auto-coding vs note-taking:
- signup: first org-level event (e.g., event_name IN ('org_signup','org_created')).
- first_transcription: first summarization_jobs.status='success'.
- regular_usage: meets a weekly stability rule, e.g. WAP ≥ 2 for 2 consecutive weeks and Locked ≥ 60% over those weeks.
- WAP: Weekly Active Providers
- consecutive weeks
- Locked rate

In [ ]:
WITH params AS (
  SELECT 2::int  AS min_wap,      -- threshold: min weekly active providers
         2::int  AS min_weeks,    -- threshold: consecutive weeks
         0.60::float AS min_locked_rate
),
signup AS (
  SELECT LOWER(e.org_id)::uuid AS org_id,
         MIN(e.occurred_at AT TIME ZONE 'America/Los_Angeles') AS signup_at
  FROM events_big e
  WHERE e.event_name IN ('org_signup','org_created')
  GROUP BY 1
),
first_tx AS (
  SELECT sj.org_id,
         MIN(sj.completed_at AT TIME ZONE 'America/Los_Angeles') AS first_transcription_at
  FROM summarization_jobs sj
  WHERE sj.status = 'success' AND sj.completed_at IS NOT NULL
  GROUP BY 1
),
weekly AS (  -- per org/week: WAP & Locked rate
  SELECT
    sj.org_id,
    DATE_TRUNC('week', sj.completed_at AT TIME ZONE 'America/Los_Angeles') AS week_start,
    COUNT(DISTINCT sj.provider_id) AS wap,
    COUNT(*) FILTER (WHERE sj.patient_status = 'Locked')::float
      / NULLIF(COUNT(*),0) AS locked_rate
  FROM summarization_jobs sj
  WHERE sj.completed_at IS NOT NULL
  GROUP BY 1,2
),
streak AS (  -- find first window with min_weeks consecutive weeks meeting thresholds
  SELECT w1.org_id,
         MIN(w1.week_start) AS regular_usage_start
  FROM weekly w1
  JOIN params p ON TRUE
  WHERE NOT EXISTS (  -- require a run of min_weeks weeks all meeting thresholds
    SELECT 1
    FROM generate_series(0, p.min_weeks-1) g(offset)
    LEFT JOIN weekly w2
      ON w2.org_id = w1.org_id
     AND w2.week_start = w1.week_start + (g.offset || ' weeks')::interval
    WHERE w2.wap < p.min_wap OR w2.locked_rate < p.min_locked_rate OR w2.org_id IS NULL
  )
  GROUP BY w1.org_id
),
journey AS (
  SELECT
    o.org_id,
    o.org_name,
    s.signup_at,
    f.first_transcription_at,
    st.regular_usage_start
  FROM organizations o
  LEFT JOIN signup s USING (org_id)
  LEFT JOIN first_tx f USING (org_id)
  LEFT JOIN streak st USING (org_id)
)
SELECT
  j.*,
  EXTRACT(EPOCH FROM (j.first_transcription_at - j.signup_at))/86400.0 AS days_to_first_transcription, -- days to first conversion
  EXTRACT(EPOCH FROM (j.regular_usage_start - COALESCE(j.first_transcription_at, j.signup_at)))/86400.0 AS days_to_regular_usage, -- days to stable usage
  CASE
    WHEN j.regular_usage_start IS NOT NULL THEN 'regular_usage'
    WHEN j.first_transcription_at IS NOT NULL THEN 'first_transcription'
    WHEN j.signup_at IS NOT NULL THEN 'signup'
    ELSE 'unknown'
  END AS current_stage
FROM journey j
ORDER BY current_stage, j.signup_at NULLS LAST;


Time to first value:
- Start point: signup_at (first org creation / onboarding event).
- End point: first_transcription_at (first completed summarization job with status='success').

In [ ]:
WITH signup AS (
  SELECT
    LOWER(org_id)::uuid AS org_id,
    MIN(occurred_at) AS signup_at
  FROM events_big
  WHERE event_name IN ('org_signup','org_created')
  GROUP BY 1
),
first_value AS (
  SELECT
    org_id,
    MIN(completed_at) AS first_transcription_at
  FROM summarization_jobs
  WHERE status = 'success'
  GROUP BY 1
)
SELECT
  o.org_id,
  o.org_name,
  s.signup_at,
  f.first_transcription_at,
  EXTRACT(EPOCH FROM (f.first_transcription_at - s.signup_at)) / 86400.0
    AS days_to_first_value
FROM organizations o
LEFT JOIN signup s USING (org_id)
LEFT JOIN first_value f USING (org_id)
WHERE s.signup_at IS NOT NULL AND f.first_transcription_at IS NOT NULL
ORDER BY days_to_first_value;


# Task 3.2: Metrics Architecture

#### Design a Practical Metrics System
Key Performance Indicators (KPIs)

1. Daily Active Doctors (DAD)
Measures engagement by tracking doctors who create at least one note per day.
- Definition: Number of unique doctors creating ≥1 note today
- Purpose: Monitor daily platform usage and adoption

2. Notes per Active Doctor (NPAD)
Measures productivity over a 7-day rolling window.
- Definition: Total notes created ÷ Active doctors (7-day period)
- Purpose: Track doctor efficiency and platform stickiness

3. New Doctor Activation Rate 
Measures onboarding success within first week.
- Definition: % of new doctors creating first note within 7 days
- Purpose: Evaluate effectiveness of onboarding process

4. Quality Pass Rate
Measures AI note quality over 7-day period.
- Definition: % of notes passing quality checks (composite score ≥0.7, no hard errors)
- Purpose: Monitor AI system performance and reliability

5. Monthly Recurring Revenue (MRR)
Measures business health through subscription revenue.
- Definition: Sum of active Stripe subscription MRR across organizations
- Purpose: Track revenue growth and business sustainability


#### Design a customer health score formula using available data

Basically weighting engagement as the most important to customer health. Quality is second. Everything else last.

Customer Health Score Components

| Component   | Weight | Formula                                    |
|------------|--------|---------------------------------------------|
| Engagement | 0.40   | min(notes_last_7d / 10, 1)                 |
| Quality    | 0.25   | 1 - flag_rate_last_7d                      |
| Depth      | 0.15   | min(NPAD_doctor / 3, 1)                    |
| Reliability| 0.10   | 1 - min(error_rate_last_7d / 0.01, 1)      |
| Latency    | 0.10   | avg(SLO_hit_turnaround, SLO_hit_summarize) |

# 

# 

The final customer health score is calculated as:

$Score = 100 \times (0.40E + 0.15D + 0.25Q + 0.10R + 0.10L)$

where:
- $E$ = Engagement score
- $D$ = Depth score
- $Q$ = Quality score 
- $R$ = Reliability score
- $L$ = Latency score
# 

# 

#### Create alert thresholds (when to notify CS team vs ignore)

- CS should be notified when something is broken
- If turnaround time exceeds 10 mins(-100%)
- If summarization time exceeds 3 min
- Global DAD drops >20% vs 7-day average → CS heads-up.
- Quality Pass Rate <80% in last hour (site-wide) → page on-call.


In [ ]:
def health_score(doctor_id, window_days=7):
    notes = get_notes(doctor_id, last_n_days=window_days)
    total_notes = count(notes)
    flagged = count(filter(notes, lambda n: n.flagged_for_review is True))
    flag_rate = flagged / max(total_notes, 1)

    eng = min(total_notes / 10.0, 1.0)

    npad_doctor = total_notes
    depth = min(npad_doctor / 3.0, 1.0)

    p50_tat = percentile(notes.turnaround_seconds, 50)
    p50_sum = percentile(notes.summarize_seconds, 50)
    slo_tat = 1.0 if p50_tat <= 600 else max(0.0, 1 - (p50_tat - 600) / 600)
    slo_sum = 1.0 if p50_sum <= 60  else max(0.0, 1 - (p50_sum - 60) / 60)
    latency = (slo_tat + slo_sum) / 2.0

    reqs = get_request_stats(doctor_id, last_n_days=window_days)  # from Sentry/logs
    err_rate = reqs.errors / max(reqs.total, 1)
    reliability = 1.0 - min(err_rate / 0.01, 1.0)

    quality = 1.0 - flag_rate

    score = 100 * (0.40 * eng + 0.15 * depth + 0.25 * quality + 0.10 * reliability + 0.10 * latency)
    return clamp(score, 0, 100)

Pipeline Architecture:
Sources
- Postgres: doctors, notes
- Stripe: subscriptions, revenue
- Sentry: error events 
- Optional: logs for request counts

Ingestion
- Write stripe events via webhook into pg tables stripe_events, and stripe_subscriptions
- Write sentry events via webhook into pg table sentry_events
- notes are already in pg

Transform (dbt)
- Every 30 minutes, have a worker pull from each of these tables and compute the latest values and store them in summary tables.
- fct_notes (one row per note with metrics/flags)
daily_usage (active users, usage)
quality_rollups
revenue_mrr
reliability_15m


Serving
- Dashboard queries hit mart_* views
- Alerting jobs run SQL against marts every 5–10 min
- Alerts → PagerDuty/Slack; incidents create CS tickets
